# Download the BGC-Argo data / BGCアルゴデータをダウンロードする

### Import the module required / 必要なモジュールを読み込む
- Here, only `pandas` is required / ここでは`pandas`のみ必要

In [43]:
import pandas as pd

<a id="jump_bgc-argo"></a>
### Retrieve the latest Argo index from the website / 最新のアルゴデータインデックス(カタログ)を読み込む
- This index can also be viewed from your browser / カタログはブラウザからも閲覧できる: https://data-argo.ifremer.fr/argo_synthetic-profile_index.txt
- Note this process can take some time (for a minute or so) as the index is relatively large (MBs) and the server can be slow / カタログの容量が大きいこととサーバーが遅い時があるため、カタログを読み込むのに１分ほど時間を要することがある。
- `skiprows = 8`: Ignore the first 8 rows which are irrelevant / カタログの最初の８行は必要ないので無視する

In [18]:
data = pd.read_csv('https://data-argo.ifremer.fr/argo_synthetic-profile_index.txt',skiprows=8)
data

,file,date,latitude,longitude,ocean,profiler_type,institution,parameters,parameter_data_mode,date_update
0,aoml/1900722/profiles/SD1900722_001.nc,2.006102e+13,-40.316,73.389,I,846,AO,PRES TEMP PSAL DOXY,DDDD,20220628080801
1,aoml/1900722/profiles/SD1900722_002.nc,2.006110e+13,-40.390,73.528,I,846,AO,PRES TEMP PSAL DOXY,DDDD,20220628080813
2,aoml/1900722/profiles/SD1900722_003.nc,2.006111e+13,-40.455,73.335,I,846,AO,PRES TEMP PSAL DOXY,DDDD,20220628080826
3,aoml/1900722/profiles/SD1900722_004.nc,2.006112e+13,-40.134,73.080,I,846,AO,PRES TEMP PSAL DOXY,DDDD,20220628080839
4,aoml/1900722/profiles/SD1900722_005.nc,2.006120e+13,-39.641,73.158,I,846,AO,PRES TEMP PSAL DOXY,DDDD,20220628080852
...,...,...,...,...,...,...,...,...,...,...
314717,meds/4902688/profiles/SR4902688_021D.nc,2.024061e+13,NaN,NaN,NaN,834,ME,PRES TEMP PSAL DOXY DOWN_IRRADIANCE380 DOWN_IR...,RRRRRRRRAR,20240613235758
314718,meds/4902688/profiles/SR4902688_022.nc,2.024061e+13,57.441,-49.108,A,834,ME,PRES TEMP PSAL DOXY DOWN_IRRADIANCE380 DOWN_IR...,RRRRRRRRAR,20240613235829
314719,meds/4902688/profiles/SR4902688_022D.nc,2.024061e+13,NaN,NaN,NaN,834,ME,PRES TEMP PSAL DOXY DOWN_IRRADIANCE380 DOWN_IR...,RRRRRRRRAR,20240613235819
314720,meds/4902688/profiles/SR4902688_023.nc,2.024061e+13,57.387,-49.054,A,834,ME,PRES TEMP PSAL DOXY DOWN_IRRADIANCE380 DOWN_IR...,RRRRRRRRAR,20240614235546


### Define your study area and period / 研究対象とする領域と期間を指定する
For the area, specify the four corners of the rectangular area in longitudes (0 to 360 degs) and latitudes (-90 to 90 degs) / 領域は、四角形の四隅の経度(0〜360)と緯度(-90〜90)を指定する
- lon0: the western edge of the box
- lon1: the eastern edge of the box
- lat0: the southern edge of the box
- lat1: the western edge of the box

For the period, specify the first and last dates in the yyyymmdd format / 期間は、最初と最後の日付をyyyymmddフォーマットで指定する
- date0: the first date
- date1: the last date

In [24]:
lon0 = 0
lon1 = 360
lat0 = -85
lat1 = -65
date0 = 20200101
date1 = 20200131

### Create an index based on your study area and period / 研究対象領域と期間に該当するデータのみを含めたカタログを作成する
- note: define this even if you want for the whole region. this filtering helps remove the data with NaN values for dates.
- For example for global coverage;
datasub = data[(data['longitude'] > 0) & (data['longitude'] < 360) & (data['latitude'] > -90) & (data['latitude'] < 90) & (data['date']>0)]

In [25]:
#extract the dataset based on lon, lat, and variable (chl). Date has a weird format so extract first then postprocess dates
datasub = data[(data['longitude'] >= lon0) & (data['longitude'] <= lon1) & (data['latitude'] >= lat0) &
               (data['latitude'] <= lat1) & (data['date'] >= 0) & (data['parameters'].str.contains(' CHLA '))]
#modify the date format to be yyyymmdd
datestr = [str(num) for num in datasub['date']]
datestr = [num[0:8] for num in datestr]
dateint = [int(num) for num in datestr]
datasub.loc[:,'date'] = dateint
#refine the dataset based on the selected period
datasub = datasub[(datasub['date'].values >= date0) & (datasub['date'].values <= date1)]
datasub

,file,date,latitude,longitude,ocean,profiler_type,institution,parameters,parameter_data_mode,date_update
70131,aoml/5904857/profiles/SD5904857_110.nc,20200110.0,-68.778,179.844,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAADD,20230105132622
70132,aoml/5904857/profiles/SD5904857_111.nc,20200121.0,-68.656,179.775,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAADD,20230105132635
70133,aoml/5904857/profiles/SD5904857_112.nc,20200131.0,-68.580,179.557,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAADD,20230105132648
74861,aoml/5905102/profiles/SD5905102_100.nc,20200106.0,-67.825,165.160,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAARD,20230309225313
74862,aoml/5905102/profiles/SD5905102_101.nc,20200116.0,-67.791,165.110,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAARD,20230309225326
74863,aoml/5905102/profiles/SD5905102_102.nc,20200126.0,-67.758,165.059,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAARD,20230309225339
82911,aoml/5905638/profiles/SD5905638_066.nc,20200107.0,-69.171,171.854,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAADD,20230829123927
82912,aoml/5905638/profiles/SD5905638_067.nc,20200117.0,-69.194,171.797,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAADD,20230829123938
82913,aoml/5905638/profiles/SD5905638_068.nc,20200127.0,-69.199,171.766,P,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAADD,20230829123951
89864,aoml/5906033/profiles/SD5906033_027.nc,20200104.0,-66.637,20.564,I,846,AO,PRES TEMP PSAL DOXY CHLA BBP700 PH_IN_SITU_TOT...,DDDDAADD,20240412130858


### Download the synthetic profiles / 該当するプロファイルを全てダウンロードする
- `-N` means to download only if the file has been updated / `-N`はファイルが更新されている場合のみダウンロードする（再実行する時に便利）

In [41]:
for i in range(datasub.shape[0]):
    url = 'ftp://ftp.ifremer.fr/ifremer/argo/dac/'+datasub['file'].values[i]
    !wget -N {url} -P data-argo

--2024-06-20 09:43:51--  ftp://ftp.ifremer.fr/ifremer/argo/dac/aoml/5904857/profiles/SD5904857_110.nc
           => 'data-argo/.listing'
Resolving ftp.ifremer.fr (ftp.ifremer.fr)... 134.246.142.21
Connecting to ftp.ifremer.fr (ftp.ifremer.fr)|134.246.142.21|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /ifremer/argo/dac/aoml/5904857/profiles ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [    <=>             ]  34.86K  38.3KB/s    in 0.9s    

2024-06-20 09:43:55 (38.3 KB/s) - 'data-argo/.listing' saved [35700]

Removed 'data-argo/.listing'.
--2024-06-20 09:43:55--  ftp://ftp.ifremer.fr/ifremer/argo/dac/aoml/5904857/profiles/SD5904857_110.nc
           => 'data-argo/SD5904857_110.nc'
==> CWD not required.
==> PASV ... done.    ==> RETR SD5904857_110.nc ... done.
Length: 175584 (171K)

SD5904857_110.nc    100%[===================>] 171.47K  79.3KB/s    in 2.2s    

2024-06-2

### That's it! You should have the profiles in your `echoes-ss2024/data-argo` directory / おしまい！`echoes-ss2024/data-argo`というディレクトリにプロファイルが全て保存されているはず